# Notebook 5: Interfaz Interactiva con Gradio

Este notebook proporciona una interfaz web interactiva para buscar partidas:
- Búsqueda en tiempo real
- Ajuste del número de resultados
- Visualización clara de estadísticas
- Resultados con similitud y precios

**Flujo**: Interfaz Web → Búsqueda → Resultados Formateados

## 1. Importar Librerías

In [4]:
pip install gradio


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import chromadb
from chromadb.utils import embedding_functions
from pathlib import Path
import gradio as gr
import pandas as pd

## 2. Configuración y Conexión a ChromaDB

In [6]:
BASE_DIR = Path.cwd()
CHROMA_DIR = BASE_DIR / "chroma_propaher_db"
COLLECTION_NAME = "partidas_propaher"
EMBEDDING_MODEL = "paraphrase-multilingual-MiniLM-L12-v2"

client = chromadb.PersistentClient(path=str(CHROMA_DIR))

embedding_fn = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=EMBEDDING_MODEL
)

collection = client.get_collection(
    name=COLLECTION_NAME,
    embedding_function=embedding_fn
)

print(f"Conectado a ChromaDB")
print(f"Colección: {collection.name}")
print(f"Total documentos: {collection.count()}")

Loading weights: 100%|██████████| 199/199 [00:00<00:00, 1666.47it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Conectado a ChromaDB
Colección: partidas_propaher
Total documentos: 412


## 3. Función de Búsqueda

In [7]:
def buscar(query, n_resultados=5):
    if not query or query.strip() == "":
        return None, None
    
    resultados = collection.query(
        query_texts=[query],
        n_results=n_resultados
    )
    
    partidas = []
    precios = []
    similitudes = []
    
    for meta, dist in zip(
        resultados["metadatas"][0],
        resultados["distances"][0]
    ):
        similitud = 1 - dist
        precio = meta["precio_unitario"]
        precios.append(precio)
        similitudes.append(similitud)
        
        partidas.append({
            "concepto": meta["concepto_base"],
            "precio": precio,
            "unidad": meta["unidad"],
            "similitud": similitud,
            "origen": meta["origen"],
            "capitulo": meta["capitulo"]
        })
    
    precio_medio = sum(precios) / len(precios) if precios else 0
    precios_sorted = sorted(precios)
    n = len(precios_sorted)
    precio_mediana = (precios_sorted[n//2] if n % 2 == 1 
                      else (precios_sorted[n//2-1] + precios_sorted[n//2]) / 2) if precios_sorted else 0
    
    precio_ponderado = (sum(p * s for p, s in zip(precios, similitudes)) / 
                        sum(similitudes)) if similitudes and sum(similitudes) > 0 else 0
    
    stats = {
        "precio_medio": precio_medio,
        "precio_mediana": precio_mediana,
        "precio_ponderado": precio_ponderado,
        "precio_min": min(precios) if precios else 0,
        "precio_max": max(precios) if precios else 0,
        "total_resultados": len(partidas)
    }
    
    return partidas, stats

## 4. Función para Formatear Resultados

In [8]:
def formatear_resultados(partidas, stats):
    if not partidas:
        return "No se encontraron resultados.", None
    
    texto_estadisticas = f"""
### ESTADÍSTICAS DE PRECIO

- **Precio Medio (Promedio)**: {stats['precio_medio']:.2f}€
- **Precio Mediana**: {stats['precio_mediana']:.2f}€
- **Precio Ponderado (por similitud)**: {stats['precio_ponderado']:.2f}€
- **Precio Mínimo**: {stats['precio_min']:.2f}€
- **Precio Máximo**: {stats['precio_max']:.2f}€
- **Total resultados**: {stats['total_resultados']}
"""
    
    df_data = []
    for i, p in enumerate(partidas):
        df_data.append({
            "#": i + 1,
            "Similitud": f"{p['similitud']:.0%}",
            "Concepto": p['concepto'][:80],
            "Precio": f"{p['precio']:.2f}€",
            "Unidad": p['unidad'],
            "Capítulo": p['capitulo'][:40],
            "Origen": p['origen']
        })
    
    df = pd.DataFrame(df_data)
    
    return texto_estadisticas, df

## 5. Función Principal para Gradio

In [9]:
def buscar_partidas_gradio(query, n_resultados):
    if not query or query.strip() == "":
        return "Por favor, introduce una búsqueda.", None
    
    partidas, stats = buscar(query, n_resultados=int(n_resultados))
    
    if not partidas:
        return "No se encontraron resultados.", None
    
    texto_stats, df = formatear_resultados(partidas, stats)
    
    return texto_stats, df

## 6. Crear Interfaz Gradio

In [10]:
ejemplos = [
    ["tubo corrugado M20", 5],
    ["instalación eléctrica vivienda", 5],
    ["bandeja portacables", 3],
    ["cable RZ1", 5],
    ["cuadro eléctrico", 5],
    ["punto de luz", 5],
    ["mecanismo conmutador", 5]
]

with gr.Blocks(title="ProPaHer - Búsqueda de Precios") as demo:
    gr.Markdown(
        """
        # ProPaHer - Sistema RAG de Búsqueda de Precios
        
        Busca partidas de construcción usando búsqueda semántica.
        El sistema encuentra las partidas más similares y calcula estadísticas de precio.
        """
    )
    
    with gr.Row():
        with gr.Column(scale=3):
            query_input = gr.Textbox(
                label="Búsqueda",
                placeholder="Ej: tubo corrugado M20, cable RZ1, punto de luz...",
                lines=2
            )
        with gr.Column(scale=1):
            n_resultados_input = gr.Slider(
                minimum=1,
                maximum=10,
                value=5,
                step=1,
                label="Número de resultados"
            )
    
    buscar_btn = gr.Button("Buscar", variant="primary")
    
    gr.Markdown("### Ejemplos de búsqueda")
    gr.Examples(
        examples=ejemplos,
        inputs=[query_input, n_resultados_input],
        label="Haz click en un ejemplo para probarlo"
    )
    
    stats_output = gr.Markdown(label="Estadísticas")
    resultados_output = gr.Dataframe(
        label="Resultados",
        wrap=True,
        interactive=False
    )
    
    buscar_btn.click(
        fn=buscar_partidas_gradio,
        inputs=[query_input, n_resultados_input],
        outputs=[stats_output, resultados_output]
    )
    
    query_input.submit(
        fn=buscar_partidas_gradio,
        inputs=[query_input, n_resultados_input],
        outputs=[stats_output, resultados_output]
    )

print("Iniciando interfaz Gradio...")
print("Se abrirá en tu navegador automáticamente.")
print("Para detener, presiona el botón 'Stop' en la esquina superior derecha de Jupyter.")

demo.launch(share=False, server_port=7860)

Iniciando interfaz Gradio...
Se abrirá en tu navegador automáticamente.
Para detener, presiona el botón 'Stop' en la esquina superior derecha de Jupyter.
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
